In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
icm_sheet = pd.read_excel("icm.xlsx", engine="openpyxl")
icm_sheet = icm_sheet[['UID','RequestID','Department','BudgetItem','Amount','AmountRemarks','Vendor']]
icm_sheet.rename(columns={'UID': 'ICM UID',
                          'Department': 'ICM Department',
                          'Amount': 'ICM Amount',
                          'AmountRemarks': 'ICM Amount Remark',
                          'Vendor': 'ICM Vendor',
                          'BudgetItem' : 'ICM Budget Item'}, 
                 inplace=True)

icm_sheet.tail()

,ICM UID,RequestID,ICM Department,ICM Budget Item,ICM Amount,ICM Amount Remark,ICM Vendor
1,4240035449,4240035449,RDD-SBI-Gram-Seva-2022-23,Y-TRA-Program Manager,1487,Traveling expenses for the month of March 2025...,Ganesh Dattatray Bhandalkar
2,4240035446,4240035446,RDD-SBI-Gram-Seva-2022-23,Y-TRA-Livelihood Officer,2301,Traveling expenses for the month of March 2025...,Satej Yashwant Kurle
3,4240035444,4240035444,RDD-SBI-Gram-Seva-2022-23,Y-TRA-Convergence Officer,2029,Traveling expenses for the month of March 2025...,Maruti Bhagwat Panchal
4,4240034792,4240034792,RDD-SBI-Gram-Seva-2022-23,Y-DIGI-Gram Seva Kendra,250,Computer and CPU Servicing Charges for GSK Off...,Vipul Manohar Patil(United Solution)
5,4240034527,4240034527,RDD-SBI-Gram-Seva-2022-23,"Y-COMMUNITY EGMT-Networking, Meeting & Awareness",3850,Food Expenses for Networking Awareness Meeting...,Shree Datta Krupa Hotel


In [ ]:
tally_jv_sheet = pd.read_excel("tally.xlsx", skiprows=6, engine="openpyxl") 

tally_jv_sheet['Date'] = tally_jv_sheet['Date'].apply(lambda x: np.nan if pd.to_datetime(x, errors='coerce') is not pd.NaT else x)

# Rename column to 'Vendor'
tally_jv_sheet.rename(columns={'Date': 'Tally Vendor'}, inplace=True)

tally_jv_sheet = tally_jv_sheet.drop(index=tally_jv_sheet.index[0]).reset_index(drop=True)


tally_jv_sheet.head()

,Tally Vendor,Particulars,Vch Type,Vch No.,Debit,Credit,Unnamed: 6
0,NaN,"COMMUNITY EGMT-Networking, Meeting & Awareness",NaN,NaN,Journal,3847,3850.0
1,NaN,RDD-SBI GRAM SEVA,3850.0,Dr,NaN,NaN,NaN
2,Shree Datta Krupa Hotel,NaN,3850.0,NaN,NaN,NaN,NaN
3,NaN,Request ID :ST/RD/2405454 (UID: 4240034527)Foo...,NaN,NaN,NaN,NaN,NaN
4,NaN,DIGI-Gram Seva Kendra,NaN,NaN,Journal,3854,250.0


In [11]:
tally_jv_sheet = tally_jv_sheet[~tally_jv_sheet.astype(str).apply(lambda row: row.str.contains("Professional Tax", na=False)).any(axis=1)]
tally_jv_sheet = tally_jv_sheet[~tally_jv_sheet.astype(str).apply(lambda row: row.str.contains("Provident Fund Employee", na=False)).any(axis=1)]
tally_jv_sheet = tally_jv_sheet.reset_index(drop=True)
tally_jv_sheet.head()

,Tally Vendor,Particulars,Vch Type,Vch No.,Debit,Credit,Unnamed: 6
0,NaN,"COMMUNITY EGMT-Networking, Meeting & Awareness",NaN,NaN,Journal,3847,3850.0
1,NaN,RDD-SBI GRAM SEVA,3850.0,Dr,NaN,NaN,NaN
2,Shree Datta Krupa Hotel,NaN,3850.0,NaN,NaN,NaN,NaN
3,NaN,Request ID :ST/RD/2405454 (UID: 4240034527)Foo...,NaN,NaN,NaN,NaN,NaN
4,NaN,DIGI-Gram Seva Kendra,NaN,NaN,Journal,3854,250.0


In [12]:
# Create group numbers (each group of 4 rows)
tally_jv_sheet['Group'] = tally_jv_sheet.index // 4

# Merge using groupby (aggregating each column as a list)
tally_jv_sheet_merged = tally_jv_sheet.groupby('Group', as_index=False).agg(lambda x: list(x))

# Drop 'Group' column if not needed
tally_jv_sheet_merged.drop(columns=['Group'], inplace=True)

# Remove NaN values from lists in each column
tally_jv_sheet_merged = tally_jv_sheet_merged.apply(lambda col: col.map(lambda x: [i for i in x if pd.notna(i)]))

tally_jv_sheet_merged.head()


,Tally Vendor,Particulars,Vch Type,Vch No.,Debit,Credit,Unnamed: 6
0,[Shree Datta Krupa Hotel],"[COMMUNITY EGMT-Networking, Meeting & Awarenes...","[3850.0, 3850.0]",[Dr],[Journal],[3847],[3850.0]
1,[UNITED SOLUTIONS COMPUTER SALES & SERVICE],"[DIGI-Gram Seva Kendra, RDD-SBI GRAM SEVA, Req...","[250.0, 250.0]",[Dr],[Journal],[3854],[250.0]
2,[Ganesh Dattatray Bhandalkar],"[TRA-Program Manager, RDD-SBI GRAM SEVA, Reque...","[1487.0, 1487.0]",[Dr],[Journal],[4005],[1487.0]
3,[Satej Yashwant Kurle],"[TRA-Livelihood Officer, RDD-SBI GRAM SEVA, Re...","[2301.0, 2301.0]",[Dr],[Journal],[4006],[2301.0]
4,[Maruti Bhagwat Panchal],"[TRA-Convergence Officer, RDD-SBI GRAM SEVA, R...","[2029.0, 2029.0]",[Dr],[Journal],[4007],[2029.0]


In [13]:
# Find max list length in 'Particulars' and 'Vch Type' to create new columns
max_length = max(tally_jv_sheet_merged['Particulars'].apply(len).max(), 
                 tally_jv_sheet_merged['Vch Type'].apply(len).max())

# Create new columns dynamically
for i in range(max_length):
    tally_jv_sheet_merged[f'Particulars_{i+1}'] = tally_jv_sheet_merged['Particulars'].apply(lambda x: x[i] if i < len(x) else np.nan)
    tally_jv_sheet_merged[f'Vch Type_{i+1}'] = tally_jv_sheet_merged['Vch Type'].apply(lambda x: x[i] if i < len(x) else np.nan)

# Drop original columns
tally_jv_sheet_merged.drop(columns=['Particulars', 'Vch Type'], inplace=True)

tally_jv_sheet_merged.tail()

,Tally Vendor,Vch No.,Debit,Credit,Unnamed: 6,Particulars_1,Vch Type_1,Particulars_2,Vch Type_2,Particulars_3,Vch Type_3
1,[UNITED SOLUTIONS COMPUTER SALES & SERVICE],[Dr],[Journal],[3854],[250.0],DIGI-Gram Seva Kendra,250.0,RDD-SBI GRAM SEVA,250.0,Request ID :ST/RD/2405475 (UID: 4240034792)Com...,NaN
2,[Ganesh Dattatray Bhandalkar],[Dr],[Journal],[4005],[1487.0],TRA-Program Manager,1487.0,RDD-SBI GRAM SEVA,1487.0,Request ID :ST/RD/2405665 (UID: 4240035449)Tra...,NaN
3,[Satej Yashwant Kurle],[Dr],[Journal],[4006],[2301.0],TRA-Livelihood Officer,2301.0,RDD-SBI GRAM SEVA,2301.0,Request ID :ST/RD/2405664 (UID: 4240035446)Tra...,NaN
4,[Maruti Bhagwat Panchal],[Dr],[Journal],[4007],[2029.0],TRA-Convergence Officer,2029.0,RDD-SBI GRAM SEVA,2029.0,Request ID :ST/RD/2405663 (UID: 4240035444)Tra...,NaN
5,[Total:],[],[9917],[],[],NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
tally_jv_sheet_merged = tally_jv_sheet_merged[['Tally Vendor','Particulars_1','Vch Type_2','Particulars_2','Particulars_3']]
tally_jv_sheet_merged = tally_jv_sheet_merged.iloc[:-1]
tally_df = tally_jv_sheet_merged.rename(columns={'Particulars_1':'Tally Budget Item','Vch Type_2':'Tally Amount','Particulars_2':'Tally Department','Particulars_3':'Tally Amount Remarks'})
tally_df['Tally Vendor'] = tally_df['Tally Vendor'].apply(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x)
tally_df['Tally UID'] = tally_df['Tally Amount Remarks'].apply(
    lambda x: re.search(r'UID:\s*(\d{10})', str(x)).group(1) if isinstance(x, str) and re.search(r'UID:\s*(\d{10})', str(x)) else None
)
tally_df['Tally Amount Remarks'] = tally_df['Tally Amount Remarks'].astype(str).str[44:]
# Convert to nullable integer type
tally_df['Tally UID'] = tally_df['Tally UID'].astype('Int64')
tally_df.tail()

,Tally Vendor,Tally Budget Item,Tally Amount,Tally Department,Tally Amount Remarks,Tally UID
0,Shree Datta Krupa Hotel,"COMMUNITY EGMT-Networking, Meeting & Awareness",3850.0,RDD-SBI GRAM SEVA,ood Expenses for Networking Awareness Meeting ...,4240034527
1,UNITED SOLUTIONS COMPUTER SALES & SERVICE,DIGI-Gram Seva Kendra,250.0,RDD-SBI GRAM SEVA,omputer and CPU Servicing Charges for GSK Offi...,4240034792
2,Ganesh Dattatray Bhandalkar,TRA-Program Manager,1487.0,RDD-SBI GRAM SEVA,raveling expenses for the month of March 2025....,4240035449
3,Satej Yashwant Kurle,TRA-Livelihood Officer,2301.0,RDD-SBI GRAM SEVA,raveling expenses for the month of March 2025....,4240035446
4,Maruti Bhagwat Panchal,TRA-Convergence Officer,2029.0,RDD-SBI GRAM SEVA,raveling expenses for the month of March 2025 ...,4240035444


In [15]:
merged_df = icm_sheet.merge(tally_df, left_on="ICM UID", right_on="Tally UID", how="outer")
merged_df = merged_df[['ICM UID','Tally UID','ICM Department','Tally Department','ICM Budget Item','Tally Budget Item',	'ICM Amount','Tally Amount','ICM Amount Remark','Tally Amount Remarks','ICM Vendor','Tally Vendor']]
merged_df = merged_df.dropna()

# Define column pairs to compare
columns_to_compare = [
    ('ICM UID', 'Tally UID'),
    ('ICM Department', 'Tally Department'),
    ('ICM Budget Item', 'Tally Budget Item'),
    ('ICM Amount', 'Tally Amount'),
    ('ICM Amount Remark', 'Tally Amount Remarks'),
    ('ICM Vendor', 'Tally Vendor')
]

# Function to check matching columns
def check_match(row):
    matched = []
    not_matched = []
    
    for icm_col, tally_col in columns_to_compare:
        if row[icm_col] == row[tally_col]:
            matched.append(icm_col)
        else:
            not_matched.append(icm_col)
    
    return pd.Series([
        'Match' if not not_matched else 'Not Match',
        ', '.join(matched) if matched else 'None',
        ', '.join(not_matched) if not_matched else 'None'
    ])

# Apply function to DataFrame
merged_df[['Match_Status', 'Matched_Columns', 'Not_Matched_Columns']] = merged_df.apply(check_match, axis=1)

merged_df = merged_df[[
    "Match_Status","Matched_Columns", "Not_Matched_Columns", "ICM Budget Item", "Tally Budget Item",
    "ICM Vendor", "Tally Vendor", "ICM Amount Remark", "Tally Amount Remarks",
    "ICM UID", "Tally UID", "ICM Department", "Tally Department",
    "ICM Amount", "Tally Amount"
]]
merged_df


,Match_Status,Matched_Columns,Not_Matched_Columns,ICM Budget Item,Tally Budget Item,ICM Vendor,Tally Vendor,ICM Amount Remark,Tally Amount Remarks,ICM UID,Tally UID,ICM Department,Tally Department,ICM Amount,Tally Amount
0,Not Match,"ICM UID, ICM Amount, ICM Vendor","ICM Department, ICM Budget Item, ICM Amount Re...","Y-COMMUNITY EGMT-Networking, Meeting & Awareness","COMMUNITY EGMT-Networking, Meeting & Awareness",Shree Datta Krupa Hotel,Shree Datta Krupa Hotel,Food Expenses for Networking Awareness Meeting...,ood Expenses for Networking Awareness Meeting ...,4240034527,4240034527,RDD-SBI-Gram-Seva-2022-23,RDD-SBI GRAM SEVA,3850,3850.0
1,Not Match,"ICM UID, ICM Amount","ICM Department, ICM Budget Item, ICM Amount Re...",Y-DIGI-Gram Seva Kendra,DIGI-Gram Seva Kendra,Vipul Manohar Patil(United Solution),UNITED SOLUTIONS COMPUTER SALES & SERVICE,Computer and CPU Servicing Charges for GSK Off...,omputer and CPU Servicing Charges for GSK Offi...,4240034792,4240034792,RDD-SBI-Gram-Seva-2022-23,RDD-SBI GRAM SEVA,250,250.0
2,Not Match,"ICM UID, ICM Amount, ICM Vendor","ICM Department, ICM Budget Item, ICM Amount Re...",Y-TRA-Convergence Officer,TRA-Convergence Officer,Maruti Bhagwat Panchal,Maruti Bhagwat Panchal,Traveling expenses for the month of March 2025...,raveling expenses for the month of March 2025 ...,4240035444,4240035444,RDD-SBI-Gram-Seva-2022-23,RDD-SBI GRAM SEVA,2029,2029.0
3,Not Match,"ICM UID, ICM Amount, ICM Vendor","ICM Department, ICM Budget Item, ICM Amount Re...",Y-TRA-Livelihood Officer,TRA-Livelihood Officer,Satej Yashwant Kurle,Satej Yashwant Kurle,Traveling expenses for the month of March 2025...,raveling expenses for the month of March 2025....,4240035446,4240035446,RDD-SBI-Gram-Seva-2022-23,RDD-SBI GRAM SEVA,2301,2301.0
4,Not Match,"ICM UID, ICM Amount","ICM Department, ICM Budget Item, ICM Amount Re...",Y-TRA-Program Manager,TRA-Program Manager,Ganesh Dattatray Bhandalkar,Ganesh Dattatray Bhandalkar,Traveling expenses for the month of March 2025...,raveling expenses for the month of March 2025....,4240035449,4240035449,RDD-SBI-Gram-Seva-2022-23,RDD-SBI GRAM SEVA,1487,1487.0


In [16]:
# Get current date and time in DD-MM-YYYY_HH:MM:SS format
timestamp = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

# Generate filename with timestamp
filename = f"ICM Tally Entry Check Sheet_{timestamp}.xlsx"

# Save the DataFrame
merged_df.to_excel(filename, index=False, engine="openpyxl")

